In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.layers import Input, Bidirectional, LSTM, Dense, LayerNormalization, Dropout, Lambda
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam
import tensorflow as tf

In [2]:
# Laste poll-of-polls data
url = "https://raw.githubusercontent.com/jensmorten/onesixtynine/main/data/pollofpolls_master.csv"
df = pd.read_csv(url)

In [3]:
# Convert to datetime and set the date to the end of the month
df["Mnd"] = pd.to_datetime(df["Mnd"])

In [4]:
# Sort values and set index
df = df.sort_values("Mnd")
df.set_index("Mnd", inplace=True)
df.index.to_period('M').to_timestamp('M')

DatetimeIndex(['2008-01-31', '2008-02-29', '2008-03-31', '2008-04-30',
               '2008-05-31', '2008-06-30', '2008-07-31', '2008-08-31',
               '2008-09-30', '2008-10-31',
               ...
               '2025-01-31', '2025-02-28', '2025-03-31', '2025-04-30',
               '2025-05-31', '2025-06-30', '2025-07-31', '2025-08-31',
               '2025-09-30', '2025-10-31'],
              dtype='datetime64[ns]', name='Mnd', length=214, freq='ME')

In [5]:
df_en=df[["Ap","Hoyre","Frp","SV","SP","KrF","Venstre","MDG","Rodt", "Andre"]]

In [6]:
df

,Ap,Hoyre,Frp,SV,SP,KrF,Venstre,MDG,Rodt,Andre
Mnd,,,,,,,,,,
2008-01-31,29.3,17.2,23.9,7.4,6.1,6.4,6.5,0.0,1.3,0.0
2008-02-29,29.0,17.3,25.2,6.7,5.9,6.3,6.6,0.0,1.3,0.0
2008-03-31,28.7,18.1,25.2,7.0,5.6,6.1,6.3,0.0,1.1,0.0
2008-04-30,29.0,16.9,25.4,6.5,5.5,7.0,6.8,0.0,1.2,0.0
2008-05-31,28.9,17.8,25.9,6.7,5.7,6.2,6.2,0.0,1.4,0.0
...,...,...,...,...,...,...,...,...,...,...
2025-06-30,28.3,16.2,21.0,6.9,5.6,3.7,4.5,3.0,6.2,4.5
2025-07-31,27.7,14.8,21.5,8.2,6.3,3.2,4.7,3.5,5.9,4.1
2025-08-31,27.3,15.3,21.2,6.3,6.2,4.6,4.2,4.3,6.1,4.5


In [7]:
#n_timesteps = 5   # past steps to look at
#n_future = 12      # steps into the future we want to predict
n_features = df_en.shape[1]
series_names = df_en.columns
window_size=12

In [8]:
def windowed_dataset_multivariate(series, window_size, batch_size, shuffle_buffer):
    """
    Creates a tf.data.Dataset for multivariate time series.

    series: numpy array or tf.Tensor with shape (num_timesteps, num_features)
    window_size: number of timesteps in the input window
    batch_size: training batch size
    shuffle_buffer: buffer size for shuffling
    """
    # Make a Dataset of timesteps
    dataset = tf.data.Dataset.from_tensor_slices(series)

    # Create sliding windows of length (window_size + 1)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)

    # Convert each window into a batch tensor
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))

    # Shuffle windows
    dataset = dataset.shuffle(shuffle_buffer)

    # Split into (input, label):
    # inputs = first window_size steps, labels = last step
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))

    # Batch and prefetch for performance
    dataset = dataset.batch(batch_size).prefetch(1)

    return dataset

In [9]:
#split = int(len(X) * 0.90)
#X_train, X_test = X[:split], X[split:]
#y_train, y_test = y[:split], y[split:]

In [10]:
window_size = 24
batch_size = 100
shuffle_buffer = 100

split = int(len(df_en) * 0.90)
train_data = df_en.values[:split]
val_data   = df_en.values[split:]

train_dataset = windowed_dataset_multivariate(train_data, window_size, batch_size, shuffle_buffer=shuffle_buffer)
val_dataset   = windowed_dataset_multivariate(val_data,   window_size, batch_size, shuffle_buffer=1)


In [11]:
y_values = []
for _, y in train_dataset:
    y_values.append(y.numpy())

y_all = np.concatenate(y_values, axis=0)  # shape: (num_train, num_features)

# Compute column weights (inverse of std)
col_mean = np.mean(y_all, axis=0)
power = 1  # increase emphasis on small parties
col_weights = (1.0 / (col_mean)) ** power
##col_weights = 1.0 / (col_stds + 1e-6)  # avoid divide-by-zero
col_weights = col_weights / np.sum(col_weights)  # normalize to mean 1

print("Column weights:", col_weights)
col_weights_tf = tf.constant(col_weights, dtype=tf.float32)

Column weights: [0.01739848 0.01901967 0.03429407 0.08223436 0.05868479 0.11136478
 0.11589345 0.17043434 0.15823955 0.23243652]


In [12]:
def weighted_mse(y_true, y_pred):
    squared_error = tf.square(y_true - y_pred)
    weighted_error = squared_error * col_weights_tf  # broadcast along features
    return tf.reduce_mean(weighted_error)

In [13]:
def relative_mae(y_true, y_pred):
    abs_error = tf.abs(y_true - y_pred)/(abs(y_pred+1E-3))
    weighted_error = abs_error*100 ##* col_weights_tf  # broadcast along features
    return tf.reduce_mean(weighted_error)

In [14]:
n_features = 10
model = Sequential([ 
    tf.keras.layers.Input(shape=(window_size, n_features)), 
    #tf.keras.layers.Conv1D(filters=10, 
    #                       kernel_size=window_size, 
    #                       strides=int(np.floor(window_size/(10))), 
    #                       activation="relu", 
    #                       padding='causal'), 
    tf.keras.layers.LSTM(10, return_sequences=True), 
    tf.keras.layers.LSTM(10, return_sequences=True), 
    tf.keras.layers.LSTM(10, return_sequences=True), 
    tf.keras.layers.LSTM(10, return_sequences=True), 
    tf.keras.layers.LSTM(10, return_sequences=False), 
    tf.keras.layers.Dense(n_features),
    tf.keras.layers.Lambda(lambda x: tf.nn.softmax(x) * 100) 
])

model.compile( optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), 
              loss=relative_mae, # or "mae" if you prefer absolute error
              metrics=["mae"] )

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 24, 10)              │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 24, 10)              │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 24, 10)              │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_3 (LSTM)                        │ (None, 24, 10)              │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_4 (LSTM)                        │ (None, 10)                  │             840 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │             110 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lambda (Lambda)                      │ (None, 10)                  │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,310 (16.84 KB)

 Trainable params: 4,310 (16.84 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
callbacks = [
    EarlyStopping(monitor="mae", patience=25, restore_best_weights=True),
    ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=10, verbose=1,
        min_lr=1e-7
    )
]


history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=1000,
    callbacks=callbacks
)


Epoch 1/1000
      1/Unknown 5s 5s/step - loss: 80.5261 - mae: 8.0595

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\keras\src\trainers\epoch_iterator.py:164: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


2/2 ━━━━━━━━━━━━━━━━━━━━ 6s 698ms/step - loss: 79.2113 - mae: 7.9365 - learning_rate: 0.0010
Epoch 2/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - loss: 77.7705 - mae: 7.8463 - learning_rate: 0.0010
Epoch 3/1000
1/2 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - loss: 76.6619 - mae: 7.7745

C:\Users\jens.nilsen\python\WPy64-31230\python-3.12.3.amd64\Lib\site-packages\keras\src\callbacks\callback_list.py:145: UserWarning: Learning rate reduction is conditioned on metric `val_loss` which is not available. Available metrics are: loss,mae,learning_rate.
  callback.on_epoch_end(epoch, logs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 76.3460 - mae: 7.7520 - learning_rate: 0.0010
Epoch 4/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - loss: 74.8556 - mae: 7.6466 - learning_rate: 0.0010
Epoch 5/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 73.1863 - mae: 7.5221 - learning_rate: 0.0010
Epoch 6/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 71.3362 - mae: 7.3749 - learning_rate: 0.0010
Epoch 7/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - loss: 69.2841 - mae: 7.2010 - learning_rate: 0.0010
Epoch 8/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 67.0300 - mae: 6.9987 - learning_rate: 0.0010
Epoch 9/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - loss: 64.6857 - mae: 6.7721 - learning_rate: 0.0010
Epoch 10/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - loss: 62.3173 - mae: 6.5260 - learning_rate: 0.0010
Epoch 11/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 59.8946 - mae: 6.2592 - learning_rate: 0.0010
Epoch 12/1000
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step - loss: 57.511

In [16]:
# --- 1) Collect validation data into arrays ---
X_test, y_test = [], []
for X_batch, y_batch in val_dataset:
    X_test.append(X_batch.numpy())
    y_test.append(y_batch.numpy())

X_test = np.concatenate(X_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

print("X_test shape:", X_test.shape)  # (num_test, window_size, 10)
print("y_test shape:", y_test.shape)  # (num_test, 10)

ValueError: need at least one array to concatenate

In [ ]:
y_pred = model.predict(X_test)  # shape: (num_test, 9)

import matplotlib.pyplot as plt

plt.figure(figsize=(15, 10))
for i, col in enumerate(df_en.columns):
    mae = np.mean(np.abs(y_test[:, i] - y_pred[:, i]))
    # Compute R² (fixed parentheses)
    ss_res = np.sum((y_test[:, i] - y_pred[:, i]) ** 2)
    ss_tot = np.sum((y_test[:, i] - np.mean(y_test[:, i])) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan
    plt.subplot(3, 4, i+1)
    plt.plot(y_test[:, i], label='Actual')
    plt.plot(y_pred[:, i], label='Predicted')
    plt.title(f"{col}\nMAE: {mae:.3f} | R²: {r2:.3f}")
    plt.xlabel('Time step')
    plt.ylabel('Value')
    plt.legend()
plt.tight_layout()
plt.show()


In [ ]:
np.sum(y_test, axis=1)

In [ ]:
last_window = df_en.values[-window_size:]

In [ ]:
last_window

In [ ]:
def forecast_future(model, last_window, steps):
    """
    Predict future steps recursively using the trained model.
    model: trained keras model
    last_window: np.array (window_size, n_features)
    steps: number of timesteps to predict
    """
    preds = []
    window = last_window.copy()

    for step in range(steps):
        # Expand to batch size 1 (same shape as during training)
        x_input = np.expand_dims(window, axis=0)  # shape (1, window_size, n_features)

        # Model prediction
        y_pred = model.predict(x_input, verbose=0)[0]  # (n_features,)
        preds.append(y_pred)

        # Slide window forward: drop oldest row, append prediction
        window = np.vstack([window[1:], y_pred])

    return np.array(preds)  # shape (steps, n_features)


In [ ]:
future_steps = 12
future_preds = forecast_future(model, last_window, steps=future_steps)

print("Shape:", future_preds.shape)
# => (12, n_features)


In [ ]:

backtest_window = df_en.values[:split][-window_size:]
y_true = df_en.values[split:split+len(val_data)]  # true values for 2024

y_pred = forecast_future(model, backtest_window, steps=len(val_data))

# Compare with real 2024 data
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_true, y_pred)
print(f"Backtest MAE: {mae:.4f}")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# --- Step 1: Backtest ---
backtest_window = df_en.values[:split][-window_size:]
y_test = df_en.values[split:]  # true values for 2024
y_pred_backtest = forecast_future(model, backtest_window, steps=len(y_test))

# --- Step 2: Future Forecast ---
last_window = df_en.values[-window_size:]
future_steps = 12
y_pred_future = forecast_future(model, last_window, steps=future_steps)

# --- Step 3: Plotting ---
plt.figure(figsize=(15, 10))
for i, col in enumerate(df_en.columns):
    # Compute metrics for backtest
    mae = np.mean(np.abs(y_test[:, i] - y_pred_backtest[:, i]))
    ss_res = np.sum((y_test[:, i] - y_pred_backtest[:, i]) ** 2)
    ss_tot = np.sum((y_test[:, i] - np.mean(y_test[:, i])) ** 2)
    r2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    plt.subplot(3, 4, i + 1)

    # Plot actual (2024)
    plt.plot(y_test[:, i], label="Actual 2024", color="black")

    # Plot backtest predictions
    plt.plot(y_pred_backtest[:, i], label="Predicted 2024", linestyle="--", color="tab:blue")

    # Plot future predictions (2025+)
    future_x = np.arange(len(y_test), len(y_test) + future_steps)
    plt.plot(future_x, y_pred_future[:, i], label="Predicted 2025+", linestyle=":", color="tab:red")

    plt.title(f"{col}\nMAE: {mae:.3f} | R²: {r2:.3f}")
    plt.xlabel("Time step")
    plt.ylabel("Value")
    plt.legend()

plt.tight_layout()
plt.show()
